In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import joblib  

In [2]:
# 1. Veri Yükleme
data = pd.read_csv('veriseti_2.csv')

# 2. Veri Ön İşleme
X = data.iloc[:, :-2].values
y_age = data.iloc[:, -2].values
y_gender = data.iloc[:, -1].values

X_train, X_test, y_age_train, y_age_test = train_test_split(X, y_age, test_size=0.2, random_state=42)
_, _, y_gender_train, y_gender_test = train_test_split(X, y_gender, test_size=0.2, random_state=42)


In [3]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Scaler'ı kaydetme
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


In [ ]:
# 3. LSTM Modeli Oluşturma ve Eğitme
def create_lstm_model():
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
model_age = create_lstm_model()
checkpoint_age = ModelCheckpoint('model_age.keras', save_best_only=True, monitor='val_loss', mode='min')
model_age.fit(X_train, y_age_train, epochs=20, batch_size=32, validation_data=(X_test, y_age_test), callbacks=[checkpoint_age])


In [ ]:
model_gender = create_lstm_model()
checkpoint_gender = ModelCheckpoint('model_gender.keras', save_best_only=True, monitor='val_loss', mode='min')
model_gender.fit(X_train, y_gender_train, epochs=20, batch_size=32, validation_data=(X_test, y_gender_test), callbacks=[checkpoint_gender])


In [4]:
import numpy as np
from tensorflow.keras.models import load_model
import joblib

# Scaler'ı yükleme
scaler = joblib.load('scaler.save')

# Test Verisi Girdisi
input_data = np.array([
-8914.65513711648,-12286.43012820513,-615.6230003949224,5406.279525139665,-794.310932585303,-7943.221293322063,-6935.297467656415,2589.4934579815035,22.28666978791521,-606.0557743302968,-5439.0628653295125,1127.2990868839695,1361.795303954015,7373.169510143493,-1665.1886318108975,-1117.6044111201961,4637.5451086542125,4987.407865040031,-2352.6237738782766,656.9024967399179,2925.6616333089846,3313.707149595687,-4713.737637835104,-439.3143633450486,7788.0580909634955,-1561.2638197489791,6925.515081506464,-550.9135918820187,5340.330984703633,4912.585018746653,3105.7844809800126,-12755.461360239164,11510.481021897811,-2508.476812419883,-12445.06396499239,499.6993779311118,450.24366275615535,7321.222112676057,135.5635632401071,-653.305459531972,-3874.9719903401087,4268.520350136578,-2945.2079009566687,-1325.8010970288801,175.94337391117355,5470.11935483871,414.4304541440707,-11090.074711316396,1758.001292200233,4411.594027411703,-498.936451654299,-3469.102677053824,10593.068345813477,6466.2547948328265,7126.135259146342,-4383.486332068529,753.0546262616866,-12176.567021276596,-868.7460165369649,5907.061628959276,-5537.860856886496,5218.433576265721,-8635.62188010899,4151.4301174168295
    ])

# Veriyi ölçeklendirme (Scaler'ı kullanarak)
input_data = scaler.transform(input_data.reshape(1, -1))

# Veriyi yeniden şekillendirme
input_data = input_data.reshape((input_data.shape[0], input_data.shape[1], 1))

# Modelleri yükleme
model_age = load_model('model_age.keras')
model_gender = load_model('model_gender.keras')

# Tahmin yapma
predicted_age = model_age.predict(input_data)
predicted_gender = model_gender.predict(input_data)

# Sonuçları yazdırma
print(f"Tahmin Edilen Yaş: {predicted_age[0][0]}")
print(f"Tahmin Edilen Cinsiyet: {'Erkek' if predicted_gender[0][0] > 0.5 else 'Kadın'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
Tahmin Edilen Yaş: 38.90289306640625
Tahmin Edilen Cinsiyet: Erkek
